[← Modules](../../../getting_started/python_to_ros/modules.rst)

## Setup: Imports and Helper Functions

In [ ]:
import numpy as np
from pykal.ros.ros2py_py2ros import (
    # Basic geometry
    ros2py_vector3, py2ros_vector3,
    ros2py_quaternion, py2ros_quaternion,
    ros2py_pose, py2ros_pose,
    ros2py_twist, py2ros_twist,
    
    # Stamped versions
    ros2py_pose_stamped, py2ros_pose_stamped,
    ros2py_twist_stamped, py2ros_twist_stamped,
    ros2py_vector3_stamped, py2ros_vector3_stamped,
    
    # Complex messages
    ros2py_odometry, py2ros_odometry,
    ros2py_imu, py2ros_imu,
    ros2py_transform_stamped, py2ros_transform_stamped,
    
    # Sensor messages
    ros2py_laserscan, py2ros_laserscan,
    ros2py_joint_state, py2ros_joint_state,
    
    # Time utilities
    ros2py_time, py2ros_time,
    ros2py_header, py2ros_header,
    
    # Registries
    ROS2PY_DEFAULT,
    PY2ROS_DEFAULT
)

# Import ROS message types
from geometry_msgs.msg import Vector3, Quaternion, Pose, Twist, PoseStamped, TwistStamped, Vector3Stamped, TransformStamped
from nav_msgs.msg import Odometry
from sensor_msgs.msg import Imu, LaserScan, JointState
from std_msgs.msg import Header
from builtin_interfaces.msg import Time

print("Imports successful!")
print(f"Default ROS2PY converters: {len(ROS2PY_DEFAULT)} message types")
print(f"Default PY2ROS converters: {len(PY2ROS_DEFAULT)} message types")

## Part 1: Time and Header Conversions

ROS uses `builtin_interfaces/Time` for timestamps and `std_msgs/Header` for metadata. We need to convert these to/from Python floats (seconds since epoch).

### Time Conversion

ROS Time has two fields:
- `sec`: Integer seconds
- `nanosec`: Nanoseconds (0-999,999,999)

We convert to a single float representing seconds.

In [ ]:
# Python time → ROS Time
t_python = 1234.567890123  # seconds with nanosecond precision
time_msg = py2ros_time(t_python)

print("Python → ROS Time:")
print(f"  Input: {t_python} seconds")
print(f"  Output: sec={time_msg.sec}, nanosec={time_msg.nanosec}")

# ROS Time → Python time
t_back = ros2py_time(time_msg)
print(f"  Roundtrip: {t_back} seconds")
print(f"  Error: {abs(t_back - t_python):.12f} seconds")

### Header Conversion

Headers contain:
- `stamp`: Time message
- `frame_id`: String identifying coordinate frame

We convert to/from a dictionary: `{"t": float, "frame_id": str}`

In [ ]:
# Create ROS Header
header_msg = py2ros_header(t=10.5, frame_id="base_link")
print("Python → ROS Header:")
print(f"  frame_id: {header_msg.frame_id}")
print(f"  stamp: {header_msg.stamp.sec}.{header_msg.stamp.nanosec:09d}")

# Convert back to Python
header_dict = ros2py_header(header_msg)
print(f"\nROS → Python Header:")
print(f"  {header_dict}")

## Part 2: Basic Geometry Messages

These are the fundamental building blocks for robotics: points, orientations, and velocities.

### Vector3: 3D Points and Directions

**Format**: `[x, y, z]` as NumPy array of shape `(3,)`

In [ ]:
# Create a 3D vector
position = np.array([1.0, 2.0, 3.0])
vec_msg = py2ros_vector3(position)

print("NumPy → ROS Vector3:")
print(f"  Input: {position}")
print(f"  Output: x={vec_msg.x}, y={vec_msg.y}, z={vec_msg.z}")

# Convert back
position_back = ros2py_vector3(vec_msg)
print(f"  Roundtrip: {position_back}")
print(f"  Shape: {position_back.shape}")

# Works with column vectors too
position_col = np.array([[1.0], [2.0], [3.0]])  # Shape (3, 1)
vec_msg2 = py2ros_vector3(position_col)
print(f"\nColumn vector → ROS: x={vec_msg2.x}, y={vec_msg2.y}, z={vec_msg2.z}")

### Quaternion: 3D Rotations

**Format**: `[x, y, z, w]` as NumPy array of shape `(4,)`

Quaternions represent rotations. The `w` component comes last (unlike some conventions).

In [ ]:
# Identity rotation (no rotation)
quat_identity = np.array([0.0, 0.0, 0.0, 1.0])  # [x, y, z, w]
quat_msg = py2ros_quaternion(quat_identity)

print("NumPy → ROS Quaternion:")
print(f"  Input: {quat_identity}")
print(f"  Output: x={quat_msg.x}, y={quat_msg.y}, z={quat_msg.z}, w={quat_msg.w}")

# 90° rotation around Z axis
quat_90z = np.array([0.0, 0.0, 0.707, 0.707])  # Approx values
quat_msg2 = py2ros_quaternion(quat_90z)
print(f"\n90° Z rotation → ROS: x={quat_msg2.x:.3f}, y={quat_msg2.y:.3f}, "
      f"z={quat_msg2.z:.3f}, w={quat_msg2.w:.3f}")

# Roundtrip
quat_back = ros2py_quaternion(quat_msg2)
print(f"  Roundtrip: {quat_back}")

### Pose: Position + Orientation

**Format**: `[px, py, pz, qx, qy, qz, qw]` as NumPy array of shape `(7,)`

Combines a 3D position with a quaternion orientation.

In [ ]:
# Robot pose: at (1, 2, 0) with identity rotation
pose_array = np.array([1.0, 2.0, 0.0,  # position
                       0.0, 0.0, 0.0, 1.0])  # quaternion
pose_msg = py2ros_pose(pose_array)

print("NumPy → ROS Pose:")
print(f"  Position: ({pose_msg.position.x}, {pose_msg.position.y}, {pose_msg.position.z})")
print(f"  Orientation: ({pose_msg.orientation.x}, {pose_msg.orientation.y}, "
      f"{pose_msg.orientation.z}, {pose_msg.orientation.w})")

# Convert back
pose_back = ros2py_pose(pose_msg)
print(f"\nROS → NumPy Pose:")
print(f"  Array: {pose_back}")
print(f"  Shape: {pose_back.shape}")

### Twist: Linear + Angular Velocity

**Format**: `[vx, vy, vz, ωx, ωy, ωz]` as NumPy array of shape `(6,)`

Represents velocity (linear and angular) of a rigid body.

In [ ]:
# Moving forward at 0.5 m/s, turning at 0.2 rad/s
twist_array = np.array([0.5, 0.0, 0.0,  # linear velocity
                        0.0, 0.0, 0.2])  # angular velocity
twist_msg = py2ros_twist(twist_array)

print("NumPy → ROS Twist:")
print(f"  Linear: ({twist_msg.linear.x}, {twist_msg.linear.y}, {twist_msg.linear.z}) m/s")
print(f"  Angular: ({twist_msg.angular.x}, {twist_msg.angular.y}, {twist_msg.angular.z}) rad/s")

# Convert back
twist_back = ros2py_twist(twist_msg)
print(f"\nROS → NumPy Twist: {twist_back}")

## Part 3: Stamped Messages (With Time)

"Stamped" messages add a header (timestamp + frame_id) to basic geometry types. The timestamp allows synchronization across multiple sensors.

### PoseStamped: Pose + Timestamp

**Format**: `[t, px, py, pz, qx, qy, qz, qw]` as NumPy array of shape `(8,)`

In [ ]:
# Pose at time t=5.0 seconds
pose_stamped_array = np.array([5.0,  # time
                               1.0, 2.0, 3.0,  # position
                               0.0, 0.0, 0.0, 1.0])  # quaternion

pose_stamped_msg = py2ros_pose_stamped(pose_stamped_array, frame_id="world")

print("NumPy → ROS PoseStamped:")
print(f"  Timestamp: {ros2py_time(pose_stamped_msg.header.stamp):.3f} s")
print(f"  Frame: {pose_stamped_msg.header.frame_id}")
print(f"  Position: ({pose_stamped_msg.pose.position.x}, "
      f"{pose_stamped_msg.pose.position.y}, {pose_stamped_msg.pose.position.z})")

# Convert back
pose_stamped_back = ros2py_pose_stamped(pose_stamped_msg)
print(f"\nROS → NumPy PoseStamped:")
print(f"  Array: {pose_stamped_back}")
print(f"  [time, x, y, z, qx, qy, qz, qw]")

### TwistStamped: Velocity + Timestamp

**Format**: `[t, vx, vy, vz, ωx, ωy, ωz]` as NumPy array of shape `(7,)`

In [ ]:
# Velocity measurement at t=10.5
twist_stamped_array = np.array([10.5,  # time
                                0.3, 0.0, 0.0,  # linear
                                0.0, 0.0, 0.1])  # angular

twist_stamped_msg = py2ros_twist_stamped(twist_stamped_array, frame_id="base_link")

print("NumPy → ROS TwistStamped:")
print(f"  Time: {ros2py_time(twist_stamped_msg.header.stamp):.3f} s")
print(f"  Frame: {twist_stamped_msg.header.frame_id}")
print(f"  Linear vel: ({twist_stamped_msg.twist.linear.x:.2f}, "
      f"{twist_stamped_msg.twist.linear.y:.2f}, {twist_stamped_msg.twist.linear.z:.2f}) m/s")

# Roundtrip
twist_stamped_back = ros2py_twist_stamped(twist_stamped_msg)
print(f"\nRoundtrip: {twist_stamped_back}")

### Vector3Stamped: 3D Vector + Timestamp

**Format**: `[t, x, y, z]` as NumPy array of shape `(4,)`

Useful for force vectors, acceleration, etc.

In [ ]:
# Acceleration measurement
accel_array = np.array([2.5,  # time
                        0.1, -0.2, 9.81])  # acceleration (m/s²)

accel_msg = py2ros_vector3_stamped(accel_array, frame_id="imu_link")

print("NumPy → ROS Vector3Stamped:")
print(f"  Time: {ros2py_time(accel_msg.header.stamp):.3f} s")
print(f"  Vector: ({accel_msg.vector.x:.2f}, {accel_msg.vector.y:.2f}, "
      f"{accel_msg.vector.z:.2f})")

accel_back = ros2py_vector3_stamped(accel_msg)
print(f"  Roundtrip: {accel_back}")

## Part 4: Complex Robot Messages

Real robots publish richer messages combining multiple pieces of information.

### Odometry: Full Robot State

**Format**: `[px, py, pz, qx, qy, qz, qw, vx, vy, vz, ωx, ωy, ωz]` as shape `(13,)`

Combines pose (7) and twist (6) into a single message. Common output from robot base controllers.

In [ ]:
# Complete robot state
odom_array = np.array([
    # Pose: position
    1.5, 0.8, 0.0,
    # Pose: orientation (quaternion)
    0.0, 0.0, 0.383, 0.924,  # ~45° rotation around Z
    # Twist: linear velocity
    0.2, 0.1, 0.0,
    # Twist: angular velocity
    0.0, 0.0, 0.3
])

odom_msg = py2ros_odometry(odom_array, frame_id="odom", child_frame_id="base_link")

print("NumPy → ROS Odometry:")
print(f"  Frame: {odom_msg.header.frame_id} → {odom_msg.child_frame_id}")
print(f"  Position: ({odom_msg.pose.pose.position.x:.2f}, "
      f"{odom_msg.pose.pose.position.y:.2f}, {odom_msg.pose.pose.position.z:.2f})")
print(f"  Orientation: quat({odom_msg.pose.pose.orientation.x:.3f}, "
      f"{odom_msg.pose.pose.orientation.y:.3f}, {odom_msg.pose.pose.orientation.z:.3f}, "
      f"{odom_msg.pose.pose.orientation.w:.3f})")
print(f"  Linear vel: ({odom_msg.twist.twist.linear.x:.2f}, "
      f"{odom_msg.twist.twist.linear.y:.2f}, {odom_msg.twist.twist.linear.z:.2f})")
print(f"  Angular vel: ({odom_msg.twist.twist.angular.x:.2f}, "
      f"{odom_msg.twist.twist.angular.y:.2f}, {odom_msg.twist.twist.angular.z:.2f})")

# Convert back
odom_back = ros2py_odometry(odom_msg)
print(f"\nROS → NumPy Odometry:")
print(f"  Shape: {odom_back.shape}")
print(f"  Array: {odom_back}")
print(f"  [px, py, pz, qx, qy, qz, qw, vx, vy, vz, wx, wy, wz]")

### IMU: Inertial Measurement Unit

**Format**: `[qx, qy, qz, qw, ωx, ωy, ωz, ax, ay, az]` as shape `(10,)`

Contains:
- Orientation (quaternion, 4)
- Angular velocity (gyro, 3)
- Linear acceleration (accel, 3)

Note: Covariances are not included in the array representation.

In [ ]:
# IMU data from a quadrotor
imu_array = np.array([
    # Orientation (quaternion)
    0.0, 0.0, 0.0, 1.0,
    # Angular velocity (rad/s)
    0.1, -0.05, 0.2,
    # Linear acceleration (m/s²)
    0.5, 0.2, 9.81  # Gravity + small perturbation
])

imu_msg = py2ros_imu(imu_array)

print("NumPy → ROS IMU:")
print(f"  Orientation: quat({imu_msg.orientation.x:.3f}, {imu_msg.orientation.y:.3f}, "
      f"{imu_msg.orientation.z:.3f}, {imu_msg.orientation.w:.3f})")
print(f"  Gyro: ({imu_msg.angular_velocity.x:.2f}, {imu_msg.angular_velocity.y:.2f}, "
      f"{imu_msg.angular_velocity.z:.2f}) rad/s")
print(f"  Accel: ({imu_msg.linear_acceleration.x:.2f}, {imu_msg.linear_acceleration.y:.2f}, "
      f"{imu_msg.linear_acceleration.z:.2f}) m/s²")

# Roundtrip
imu_back = ros2py_imu(imu_msg)
print(f"\nRoundtrip: {imu_back}")

### TransformStamped: TF Tree Transforms

**Format**: `[t, tx, ty, tz, qx, qy, qz, qw]` as shape `(8,)`

Similar to PoseStamped but used for coordinate frame transforms in the TF tree.

In [ ]:
# Transform from base_link to laser_link
transform_array = np.array([
    15.0,  # time
    0.1, 0.0, 0.2,  # translation
    0.0, 0.0, 0.0, 1.0  # rotation (identity)
])

tf_msg = py2ros_transform_stamped(
    transform_array,
    frame_id="base_link",
    child_frame_id="laser_link"
)

print("NumPy → ROS TransformStamped:")
print(f"  Time: {ros2py_time(tf_msg.header.stamp):.3f} s")
print(f"  Transform: {tf_msg.header.frame_id} → {tf_msg.child_frame_id}")
print(f"  Translation: ({tf_msg.transform.translation.x:.2f}, "
      f"{tf_msg.transform.translation.y:.2f}, {tf_msg.transform.translation.z:.2f})")

# Roundtrip
tf_back = ros2py_transform_stamped(tf_msg)
print(f"\nRoundtrip: {tf_back}")

## Part 5: Sensor Messages

Specialized messages for common robot sensors.

### LaserScan: 2D Lidar Data

**Format**: Dictionary with `ranges` array and metadata

LaserScan contains an array of range measurements plus scan parameters.

In [ ]:
# Simulate a 180° scan with 10 beams
ranges = np.array([1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 3.5, 3.0, 2.5])

scan_msg = py2ros_laserscan(
    ranges,
    angle_min=-np.pi/2,  # -90°
    angle_max=np.pi/2,   # +90°
    angle_increment=np.pi/9,  # 180° / 10 beams
    range_min=0.1,
    range_max=10.0,
    frame_id="laser_link",
    stamp=20.0
)

print("NumPy → ROS LaserScan:")
print(f"  Frame: {scan_msg.header.frame_id}")
print(f"  Time: {ros2py_time(scan_msg.header.stamp):.3f} s")
print(f"  Angle range: {np.rad2deg(scan_msg.angle_min):.1f}° to {np.rad2deg(scan_msg.angle_max):.1f}°")
print(f"  Number of beams: {len(scan_msg.ranges)}")
print(f"  Ranges: {scan_msg.ranges[:5]}...")

# Convert back
scan_dict = ros2py_laserscan(scan_msg)
print(f"\nROS → Python Dictionary:")
print(f"  Keys: {list(scan_dict.keys())}")
print(f"  Ranges shape: {scan_dict['ranges'].shape}")
print(f"  Ranges: {scan_dict['ranges']}")

### JointState: Robot Joint Positions/Velocities

**Format**: Dictionary with `name`, `position`, `velocity`, `effort` arrays

Describes the state of all joints in a robot (e.g., arm joints, wheel joints).

In [ ]:
# 3-joint robot arm
joint_names = ["shoulder", "elbow", "wrist"]
joint_positions = np.array([0.5, 1.0, -0.3])  # radians
joint_velocities = np.array([0.1, 0.05, 0.0])  # rad/s
joint_efforts = np.array([2.0, 1.5, 0.5])  # N⋅m

joint_msg = py2ros_joint_state(
    names=joint_names,
    position=joint_positions,
    velocity=joint_velocities,
    effort=joint_efforts
)

print("NumPy → ROS JointState:")
print(f"  Joints: {joint_msg.name}")
print(f"  Positions: {joint_msg.position}")
print(f"  Velocities: {joint_msg.velocity}")
print(f"  Efforts: {joint_msg.effort}")

# Convert back
joint_dict = ros2py_joint_state(joint_msg)
print(f"\nROS → Python Dictionary:")
print(f"  Keys: {list(joint_dict.keys())}")
print(f"  Names: {joint_dict['name']}")
print(f"  Positions: {joint_dict['position']}")

## Part 6: Using the Conversion Registries

The `ROS2PY_DEFAULT` and `PY2ROS_DEFAULT` dictionaries map message types to converter functions. ROSNode uses these registries automatically.

### Checking Available Converters

In [ ]:
print("Available ROS2PY converters:")
for msg_type in sorted(ROS2PY_DEFAULT.keys(), key=lambda t: t.__name__):
    print(f"  {msg_type.__module__}.{msg_type.__name__}")

print(f"\nTotal: {len(ROS2PY_DEFAULT)} message types")

### Using Registries for Generic Conversion

In [ ]:
# Generic converter function
def convert_ros_to_numpy(msg):
    """Convert any registered ROS message to NumPy array."""
    msg_type = type(msg)
    if msg_type not in ROS2PY_DEFAULT:
        raise TypeError(f"No converter for {msg_type.__name__}")
    
    converter = ROS2PY_DEFAULT[msg_type]
    return converter(msg)

# Test with different message types
test_msgs = [
    py2ros_vector3(np.array([1, 2, 3])),
    py2ros_twist(np.array([0.5, 0, 0, 0, 0, 0.2])),
    py2ros_pose(np.array([1, 2, 3, 0, 0, 0, 1])),
]

for msg in test_msgs:
    arr = convert_ros_to_numpy(msg)
    print(f"{type(msg).__name__:20s} → {arr}")

## Part 7: Practical Workflow Examples

Real-world scenarios combining multiple conversions.

### Example 1: Processing Odometry Stream

In [ ]:
# Simulate receiving odometry messages
def process_odometry_callback(odom_msg: Odometry):
    """Extract position and velocity from odometry."""
    # Convert to NumPy
    odom_array = ros2py_odometry(odom_msg)
    
    # Extract components
    position = odom_array[0:3]  # [px, py, pz]
    quaternion = odom_array[3:7]  # [qx, qy, qz, qw]
    linear_vel = odom_array[7:10]  # [vx, vy, vz]
    angular_vel = odom_array[10:13]  # [wx, wy, wz]
    
    return {
        "position": position,
        "orientation": quaternion,
        "linear_velocity": linear_vel,
        "angular_velocity": angular_vel,
        "speed": np.linalg.norm(linear_vel)
    }

# Test with sample message
sample_odom = py2ros_odometry(
    np.array([5, 3, 0,  # position
              0, 0, 0, 1,  # orientation
              0.8, 0.2, 0,  # linear vel
              0, 0, 0.1]),  # angular vel
    frame_id="odom"
)

result = process_odometry_callback(sample_odom)
print("Processed Odometry:")
for key, value in result.items():
    if isinstance(value, np.ndarray):
        print(f"  {key}: {value}")
    else:
        print(f"  {key}: {value:.3f}")

### Example 2: Creating Stamped Messages with Current Time

In [ ]:
import time

def create_velocity_command(vx: float, vy: float, omega: float) -> TwistStamped:
    """Create a timestamped velocity command."""
    current_time = time.time()
    
    # Create array with timestamp
    cmd_array = np.array([current_time, vx, vy, 0.0, 0.0, 0.0, omega])
    
    # Convert to ROS message
    return py2ros_twist_stamped(cmd_array, frame_id="base_link")

# Generate command
cmd_msg = create_velocity_command(vx=0.5, vy=0.0, omega=0.3)

print("Velocity Command:")
print(f"  Time: {ros2py_time(cmd_msg.header.stamp):.3f} s")
print(f"  Linear: ({cmd_msg.twist.linear.x:.2f}, {cmd_msg.twist.linear.y:.2f}) m/s")
print(f"  Angular: {cmd_msg.twist.angular.z:.2f} rad/s")

### Example 3: Batch Processing Multiple Poses

In [ ]:
# Generate trajectory of poses
def generate_circular_trajectory(num_points: int, radius: float) -> list:
    """Generate poses along a circular path."""
    poses = []
    for i in range(num_points):
        angle = 2 * np.pi * i / num_points
        
        # Position along circle
        x = radius * np.cos(angle)
        y = radius * np.sin(angle)
        z = 0.0
        
        # Orientation facing tangent to circle
        qz = np.sin(angle / 2)
        qw = np.cos(angle / 2)
        
        pose_array = np.array([x, y, z, 0, 0, qz, qw])
        poses.append(py2ros_pose(pose_array))
    
    return poses

# Generate and convert
trajectory = generate_circular_trajectory(num_points=8, radius=2.0)

print(f"Generated {len(trajectory)} poses:")
for i, pose in enumerate(trajectory[:4]):  # Show first 4
    arr = ros2py_pose(pose)
    print(f"  Pose {i}: pos=({arr[0]:.2f}, {arr[1]:.2f}), "
          f"quat=({arr[3]:.3f}, {arr[4]:.3f}, {arr[5]:.3f}, {arr[6]:.3f})")
print(f"  ... and {len(trajectory)-4} more")

## Summary

You've learned all the essential ROS ↔ NumPy conversions:

**Basic Types**:
- `Vector3`: 3D vectors → `(3,)` arrays
- `Quaternion`: Rotations → `(4,)` arrays `[x, y, z, w]`
- `Pose`: Position + orientation → `(7,)` arrays
- `Twist`: Linear + angular velocity → `(6,)` arrays

**Stamped Types** (add timestamp):
- `PoseStamped` → `(8,)`: `[t, px, py, pz, qx, qy, qz, qw]`
- `TwistStamped` → `(7,)`: `[t, vx, vy, vz, wx, wy, wz]`
- `Vector3Stamped` → `(4,)`: `[t, x, y, z]`

**Complex Messages**:
- `Odometry` → `(13,)`: Full robot state (pose + twist)
- `IMU` → `(10,)`: Orientation + gyro + accel
- `TransformStamped` → `(8,)`: Coordinate frame transforms

**Sensor Messages**:
- `LaserScan` → Dictionary with ranges array + metadata
- `JointState` → Dictionary with joint arrays

**Registries**:
- `ROS2PY_DEFAULT`: Automatic ROS → NumPy conversion
- `PY2ROS_DEFAULT`: Automatic NumPy → ROS conversion

These converters are the foundation for building ROS nodes with pykal's `ROSNode` wrapper. Next, see `rosnode.ipynb` to learn how ROSNode uses these converters automatically!

[← Modules](../../../getting_started/python_to_ros/modules.rst)